In [113]:
import numpy as np
import scipy as sp
from scipy import linalg as la
from matplotlib import pyplot as plt
import cmath

In [114]:
#problem 1
def drazin_test(A, A_d, k):
    A = np.array(A, dtype = np.float)
    A_d = np.array(A_d, dtype = np.float)
    result = np.allclose([(A@A_d), (np.linalg.matrix_power(A, k+1)@A_d), (A_d@A@A_d)], [(A_d@A), (np.linalg.matrix_power(A, k)), (A_d)])
    return result
#test it
A = [[1,3,0,0], [0,1,3,0],[0,0,1,3],[0,0,0,0]]
A_d = [[1,-3,9,81],[0,1,-3,-18],[0,0,1,3],[0,0,0,0]]
k = 1
print(drazin_test(A, A_d, k))
B = [[1,1,3],[5,2,6],[-2,-1,-3]]
B_d = [[0,0,0],[0,0,0],[0,0,0]]
k = 3
print(drazin_test(B,B_d, k))


True
True


In [115]:
def index(A, tol=1e-5):
    """Compute the index of the matrix A.

    Parameters:
        A ((n,n) ndarray): An nxn matrix.

    Returns:
        k (int): The index of A.
    """

    # test for non-singularity
    if not np.allclose(la.det(A),0):
        return 0

    n = len(A)
    k = 1
    Ak = A.copy()
    while k <= n:
        r1 = np.linalg.matrix_rank(Ak)
        r2 = np.linalg.matrix_rank(np.dot(A,Ak))
        if r1 == r2:
            return k
        Ak = np.dot(A,Ak)
        k += 1

    return k


In [116]:
#problem 2
def DRAZIN(A, tol):
    A = np.array(A, dtype = np.float)
    n = A.shape[0]
    f1 = lambda x: abs(x) > tol
    f2 = lambda x: abs(x) <= tol
    Q1, S, k1 = la.schur(A, sort = f1)
    Q2, T, k2 = la.schur(A, sort = f2)
    U = np.column_stack((S[:, :k1], T[:, :(n-k1)]))
    U_inv = np.linalg.inv(U)
    V = U_inv @ A @ U
    Z =np.zeros_like(A, dtype = np.float)
    if k1 != 0:
        M_inv = np.linalg.inv(V[:k1, :k1])
        Z[:k1, :k1] = M_inv
    return (U @ Z @ U_inv)
# test case the function
A = np.random.random((9,9))
k = index(A, 1e-8)
A_d = DRAZIN(A,1e-8)
drazin_test(A, A_d, k)

True

In [117]:
# problem 3
def effective_resistance(A):
    A = np.array(A)
    n = A.shape[0]
    D = np.diag(np.sum(A, axis = 1)) # we need to calculate the Degree Matrix first
    L = D - A # create the Laplacian matrix
    R = np.zeros_like(L,dtype=np.float)
    I  = np.identity(n)
    for j in range(n):
        L_j = np.copy(L)
        L_j[j, :] = I[j, :]
        L_jd = DRAZIN(L_j, 1e-8)
        R[:, j] = np.diag(L_jd)
    np.fill_diagonal(R, 0)
    return R
    

In [119]:
A1 = [[0, 0, 1, 0],[0, 0, 0, 1],[1, 0, 0, 1],[0, 1, 1, 0]]
R1 = effective_resistance(A1)
print(R1)
A2 = [[0, 1, 1],[1, 0, 1], [1, 1, 0]]
R2 = effective_resistance(A2)
print(R2)
A3 = [[0, 2], [2,0]]
R3 = effective_resistance(A3)
print(R3)


[[ 0.  3.  1.  2.]
 [ 3.  0.  2.  1.]
 [ 1.  2.  0.  1.]
 [ 2.  1.  1.  0.]]
[[ 0.          0.66666667  0.66666667]
 [ 0.66666667  0.          0.66666667]
 [ 0.66666667  0.66666667  0.        ]]
[[ 0.   0.5]
 [ 0.5  0. ]]


In [112]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
